In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_234530/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 09:56:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 09:56:11 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Candidate table
candidate_data = {
    'id': [1, 2, 3, 4, 5],
    'name': ['A', 'B', 'C', 'D', 'E']
}

# Creating a pandas DataFrame for Candidate table
candidate_df = pd.DataFrame(candidate_data)

# Sample data for the Vote table
vote_data = {
    'id': [1, 2, 3, 4, 5],
    'candidateId': [2, 4, 3, 2, 5]
}

# Creating a pandas DataFrame for Vote table
vote_df = pd.DataFrame(vote_data)

# Displaying the Candidate DataFrame
print("Candidate table:")
print(candidate_df)

# Displaying the Vote DataFrame
print("\nVote table:")
print(vote_df)



df_person = spark.createDataFrame(candidate_df)
df_person.createOrReplaceTempView("Candidate")

df_person = spark.createDataFrame(vote_df)
df_person.createOrReplaceTempView("Vote")



Candidate table:
   id name
0   1    A
1   2    B
2   3    C
3   4    D
4   5    E

Vote table:
   id  candidateId
0   1            2
1   2            4
2   3            3
3   4            2
4   5            5


In [10]:
query = """

    select c.name from (select candidateId, vote_count from (select
        candidateId,
        count(*) as vote_count
    from
        Vote v
    group by
        v.candidateId) order by vote_count desc limit 1) t inner join Candidate c on t.candidateId = c.id
"""

In [11]:
spark.sql(query).show()

+----+
|name|
+----+
|   B|
+----+

